In [1]:
#Extract Foreground mask from the background
import cv2
import time # to introduce delay while reaading frames
import numpy as np

In [15]:
# Method 2

vidCap = cv2.VideoCapture("./day2.mp4")

ret, frame1 = vidCap.read() # reads the next frame

frame_width = int(vidCap.get(3))
frame_height = int(vidCap.get(4))

fourcc = cv2.VideoWriter_fourcc(*'MP4V')
out = cv2.VideoWriter('day2-output.mp4',fourcc, 24.0, (frame_width,frame_height))

vehicle = 0
validVehicles = []

while vidCap.isOpened():
    ret, frame2 = vidCap.read() # reads the next frame

    if not ret:
        break

    frame = frame2.copy()

    # convert the frames to grayscale
    grayA = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)
    grayB = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)

    # extract the foreground mask
    fgMask = cv2.absdiff(grayB, grayA)

    # perform image thresholding
    ret, thresh = cv2.threshold(fgMask, 30, 255, cv2.THRESH_BINARY)

    # apply image dilation
    kernel = np.ones((3,3),np.uint8)
    dilated = cv2.dilate(thresh,kernel,iterations = 1)  

    frame1 = frame2
    
    cv2.line(dilated, (0, 80),(256,80),(100, 0, 0))
    # Reference Lines for Counter 
    cv2.line(frame, (420,890), (1650,890), (0,0,255), 2) # reference line  RED
    cv2.line(frame, (420,880), (1650,880), (0,255,0), 1) #  Offset Reference line Above
    cv2.line(frame, (420,900), (1650,900), (0,255,0), 1) #  Offset Reference line Below  
    
    # extract the contours
    contours, _ = cv2.findContours(dilated, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    
    for c in contours:
        x, y, w, h = cv2.boundingRect(c)
        
        # ignore the small contours in size
        visibleVehicle = (w > 60) and (h > 60)
        if not visibleVehicle:
            continue
        
        # remove the distraction on the road; consider only the objects on ROAD
        if x > 420 and x < 1650 and y > 200:
            # draw the bounding rectangle for all contours
            cv2.rectangle(frame, (x,y), (x+w, y+h), (0,255,0), 2)
            xMid = int((x + (x+w))/2)
            yMid = int((y + (y+h))/2)
            cv2.circle(frame, (xMid,yMid),5,(0,0,255),5)

            # add all valid vehiles into List Array
            validVehicles.append((xMid,yMid))
            # cv2.waitKey(0) # debugging purpose

            for (vX, vY) in validVehicles:
                if vY > 880 and vY < 900: # adjust this for the frame jumping
                    vehicle += 1
                    validVehicles.remove((vX,vY))
                    
                    # debugging purpose
                    # cv2.putText(frame, 'Y : {}'.format(yMid), (x, y-20), cv2.FONT_HERSHEY_PLAIN, 2, (255,255,255), 2)

    cv2.putText(frame, 'Total Vehicles : {}'.format(vehicle), (1450, 150), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255,255,255), 3)
    out.write(frame)
    
    # # show the thresh and original video
    # cv2.imshow('Foreground Mask', dilated)
    # cv2.putText(frame, 'Total Vehicles : {}'.format(vehicle), (1450, 150), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255,255,255), 3)
    # cv2.imshow('Original Video', frame)
    
    # # wait for any key to be pressed
    # if cv2.waitKey(1) & 0xFF == ord('q'):
    #     break

# release video capture
cv2.destroyAllWindows()
vidCap.release()
out.release()